<a href="https://colab.research.google.com/github/ShyamSundhar1411/My-ML-Notebooks/blob/master/Natural%20Language%20Processing/Introduction_to_NLP_in_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction to NLP Fundamentals in Tensorflow

In [1]:
!nvidia-smi

Wed Feb 15 18:55:17 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    29W /  70W |      0MiB / 15360MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2023-02-15 18:55:18--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-02-15 18:55:18 (92.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
from helper_functions import plot_loss_curves,unzip_data,create_tensorboard_callback,compare_historys

## Getting a text Dataset

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-02-15 18:55:22--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.202.128, 74.125.69.128, 64.233.182.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.202.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2023-02-15 18:55:22 (145 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
unzip_data("nlp_getting_started.zip")

## Visualizing Dataset

In [6]:
import pandas as pd
train_dataset = pd.read_csv("/content/train.csv")

In [7]:
train_dataset

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
train_dataset_shuffled = train_dataset.sample(frac = 1,random_state = 42)


In [9]:
train_dataset_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [10]:
test_data  = pd.read_csv("/content/test.csv")

In [11]:
test_data

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [12]:
train_dataset.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [13]:
import random
random_index = random.randint(0,len(train_dataset)-5)
for row in train_dataset_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target = row
  print(text,target)

Sadly How Windows 10 Reveals Microsoft's Ethics Armageddon http://t.co/sTfTjCrjEa 0
@billy_hodge Aurora theater shooting trial: Gunman expected notoriety for mass murder and nothing else  http://t.co/1RPCHRu72C 1
Officials say a quarantine is in place at a Birmingham home over a possible Ebola case.

Edward Khan of the... http://t.co/qnIJ2p8Zv6 1
@Zojadelin you literally almost had a head on collision with us today on pilot knob 1
connor franta: damn sirens I hope everyone is okay. 

dan howell: can you PLEASE get MURDERED on ANOTHER STREET 1


## Split Data into Training and Validation Sets

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences,val_sentences,train_labels,val_labels = train_test_split(
    train_dataset_shuffled['text'].to_numpy(),
    train_dataset_shuffled['target'].to_numpy(),
    test_size = 0.1,
    random_state = 42
    )

In [16]:
train_sentences[:10]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk',
       '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
       'destroy the free fandom honestly',
       'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
       '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
       'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
      dtype=object)

## Tokenization and Embedding

### Text Vectorization (Tokenization)

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(
    max_tokens = 10000,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = None,
    pad_to_max_tokens = True,
  )


In [18]:
round(sum([len(i.split()) for i in train_sentences]))/len(train_sentences)

14.901036345059115

### Text Vecotrizer Variables

In [19]:
max_vocab_length = 10000
max_length = 15
text_vectorizer = TextVectorization(
    max_tokens = max_vocab_length,
    standardize = "lower_and_strip_punctuation",
    split = "whitespace",
    ngrams = None,
    output_mode = "int",
    output_sequence_length = max_length,
    pad_to_max_tokens = True,
)

In [20]:
text_vectorizer.adapt(train_sentences)

In [21]:
sample_sentence = "Flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [22]:
random_sentences = random.choice(train_sentences)
text_vectorizer([random_sentences])


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 197,   92,    8, 1151,    3,  322,   10,  129,    8,  125,  276,
         100,  118,  581,  601]])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
top_5_words

['', '[UNK]', 'the', 'a', 'in']

## Embedding Layer

In [24]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim = max_vocab_length,
      output_dim = 128,
      input_length = max_length
                                   
)
embedding

In [25]:
random_sentence = random.choice(train_sentences)
print(random_sentence)
embedded_sentence = embedding(text_vectorizer(random_sentence))
print(embedded_sentence)

#ISIS claims responsibility for Saudi suicide bombing  http://t.co/jlMICJ6jE5
tf.Tensor(
[[ 0.03224499 -0.03191995  0.00274339 ...  0.01430322 -0.02857304
   0.04840222]
 [ 0.01727625 -0.03667642 -0.00063137 ...  0.02232519  0.03149382
  -0.03264494]
 [ 0.01305476 -0.02418034  0.02845675 ... -0.02522836 -0.02286787
   0.02216414]
 ...
 [ 0.03025594 -0.03251527  0.04310305 ...  0.0371941  -0.01600007
   0.00959847]
 [ 0.03025594 -0.03251527  0.04310305 ...  0.0371941  -0.01600007
   0.00959847]
 [ 0.03025594 -0.03251527  0.04310305 ...  0.0371941  -0.01600007
   0.00959847]], shape=(15, 128), dtype=float32)


## Model 0 : Naive Bayes (Baseline)

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()), #convert words to numbers,
    ("clf",MultinomialNB()), #model the text,
])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [27]:
base_line_score = model_0.score(val_sentences,val_labels)

In [28]:
base_line_score

0.7926509186351706

In [29]:
base_line_predict = model_0.predict(val_sentences)
base_line_predict[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating functions to compare model experiments

In [30]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  model_accuracy = accuracy_score(y_true,y_pred)*100
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average = "weighted")
  model_results = {
      "accuracy":model_accuracy,
      "precision":model_precision,
      "recall":model_recall,
      "f1":model_f1
  }
  return model_results

In [31]:
base_line_results = calculate_results(val_labels,base_line_predict)
base_line_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: Simple Dense Model

In [32]:
from helper_functions import create_tensorboard_callback
SAVE_DIR = "model_logs"

In [33]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")

In [34]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [35]:
model_1.compile(loss = "binary_crossentropy",
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ['accuracy'])

In [36]:
model_1_history = model_1.fit(
    x = train_sentences,y = train_labels,
    epochs = 5,
    validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_1_dense")]
    )

Saving TensorBoard log files to: model_logs/model_1_dense/20230215-185527
Epoch 1/5
215/215 [==============================] - 22s 67ms/step - loss: 0.6367 - accuracy: 0.6452 - val_loss: 0.5721 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4669 - accuracy: 0.8278 - val_loss: 0.4736 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.3384 - accuracy: 0.8739 - val_loss: 0.4564 - val_accuracy: 0.7808
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2563 - accuracy: 0.9091 - val_loss: 0.4592 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.1968 - accuracy: 0.9337 - val_loss: 0.4724 - val_accuracy: 0.7874


In [37]:
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4724 - accuracy: 0.7874


[0.4723564088344574, 0.787401556968689]

In [38]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(762, 1)

In [39]:
model_1_pred_probs[10]

array([0.16545299], dtype=float32)

In [40]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [41]:
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0.], dtype=float32)>

In [42]:
model_1_results = calculate_results(y_true = val_labels,y_pred = model_1_preds)
model_1_results

{'accuracy': 78.74015748031496,
 'precision': 0.7914920592553047,
 'recall': 0.7874015748031497,
 'f1': 0.7846966492209201}

## Visualizing Learned Embeddings

In [43]:
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab)

10000

In [44]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [45]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]

In [46]:
embed_weights.shape

(10000, 128)

In [47]:
embed_weights

array([[ 0.09729689, -0.05862145, -0.05597332, ..., -0.01720705,
        -0.07551063, -0.07758535],
       [-0.01167135, -0.05041189, -0.03691012, ..., -0.02275691,
        -0.04386068, -0.10145486],
       [-0.00898632, -0.0539957 , -0.05710987, ..., -0.02224413,
        -0.05441892, -0.0498178 ],
       ...,
       [ 0.00983813,  0.0115326 ,  0.01952776, ...,  0.01296569,
        -0.02400242, -0.01211414],
       [ 0.08566611, -0.04116108, -0.00496479, ..., -0.04744859,
         0.01335838,  0.01895759],
       [ 0.01974747, -0.01818537, -0.00637365, ...,  0.08755216,
        -0.02376914, -0.01072451]], dtype=float32)

## Recurrent Neural Network

### Model 2: LSTM

In [48]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64,return_sequences = True)(x) # When we are stacking we need return_sequences = True
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64,activation = "relu")(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 64)
(None, 64)


In [49]:
model_2.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [50]:
model_2_history = model_2.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_2_lstm")]
            )

Saving TensorBoard log files to: model_logs/model_2_lstm/20230215-185600
Epoch 1/5
215/215 [==============================] - 15s 45ms/step - loss: 0.3752 - accuracy: 0.8343 - val_loss: 0.4746 - val_accuracy: 0.7861
Epoch 2/5
215/215 [==============================] - 3s 13ms/step - loss: 0.2520 - accuracy: 0.8994 - val_loss: 0.6063 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1727 - accuracy: 0.9346 - val_loss: 0.6917 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1299 - accuracy: 0.9508 - val_loss: 0.7163 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0967 - accuracy: 0.9623 - val_loss: 1.1786 - val_accuracy: 0.7625


In [51]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[1.8007716e-03],
       [8.7970334e-01],
       [9.9987888e-01],
       [1.3890576e-03],
       [5.5251713e-04],
       [9.9998009e-01],
       [9.2095560e-01],
       [9.9997997e-01],
       [9.9997890e-01],
       [4.0050969e-01]], dtype=float32)

In [52]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

In [53]:
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [54]:
calculate_results(val_labels,model_2_preds)

{'accuracy': 76.24671916010499,
 'precision': 0.7757932321440772,
 'recall': 0.7624671916010499,
 'f1': 0.7556889914772474}

### Model-3 GRU Powered RNN

In [55]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#If returns sequences and stackings are true
# x = layers.LSTM(64,return_sequences = True)(x)
# x = layers.GRU(64)(x)
# x = layers.Dense(64,activation = "relu")(x)
# x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation = "sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [56]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [57]:
model_3.compile(loss = "binary_crossentropy",metrics = ['accuracy'],optimizer = "adam")

In [58]:
model_3_history = model_3.fit(train_sentences,train_labels,epochs = 5,
            validation_data = (val_sentences,val_labels),
            callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_3_gru")]
            )

Saving TensorBoard log files to: model_logs/model_3_gru/20230215-185627
Epoch 1/5
215/215 [==============================] - 11s 41ms/step - loss: 0.2099 - accuracy: 0.9133 - val_loss: 0.7361 - val_accuracy: 0.7808
Epoch 2/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1000 - accuracy: 0.9639 - val_loss: 0.6498 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0758 - accuracy: 0.9720 - val_loss: 0.8715 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0585 - accuracy: 0.9747 - val_loss: 1.1208 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 2s 8ms/step - loss: 0.0497 - accuracy: 0.9777 - val_loss: 1.2899 - val_accuracy: 0.7625


In [59]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 3ms/step


array([[1.1493318e-02],
       [8.7495321e-01],
       [9.9987829e-01],
       [1.0940708e-01],
       [1.8706701e-04],
       [9.9988389e-01],
       [1.8729212e-02],
       [9.9996567e-01],
       [9.9994075e-01],
       [9.8170394e-01]], dtype=float32)

In [60]:
model_3_pred = tf.squeeze(tf.round(model_3_pred_probs))

In [61]:
model_3_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [62]:
calculate_results(model_3_pred,val_labels)

{'accuracy': 76.24671916010499,
 'precision': 0.7750993816302697,
 'recall': 0.7624671916010499,
 'f1': 0.7651117889191904}

### Model-4 Bidirectional RNN

In [64]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Bidirectional(layers.LSTM(64,return_sequences = True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
outputs = layers.Dense(1,activation = "sigmoid",)(x)
model_4 = tf.keras.Model(inputs,outputs)

In [65]:
model_4.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                             

In [66]:
model_4.compile(loss = "binary_crossentropy",optimizer = "adam",metrics = ['accuracy'])

In [70]:
mode_4_history = model_4.fit(
    train_sentences,train_labels,epochs = 5,validation_data = (val_sentences,val_labels),
    callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,experiment_name = "model_4_bidirectional")],
)

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230215-190655
Epoch 1/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0418 - accuracy: 0.9790 - val_loss: 1.4921 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0378 - accuracy: 0.9807 - val_loss: 1.5030 - val_accuracy: 0.7651
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0387 - accuracy: 0.9813 - val_loss: 1.4175 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0415 - accuracy: 0.9787 - val_loss: 1.4763 - val_accuracy: 0.7572
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0366 - accuracy: 0.9815 - val_loss: 1.6966 - val_accuracy: 0.7598


In [75]:
model_4_preds = tf.squeeze(tf.round(model_4.predict(val_sentences)))

24/24 [==============================] - 2s 5ms/step


In [76]:
calculate_results(val_labels,model_4_preds)

{'accuracy': 75.98425196850394,
 'precision': 0.7604578907479682,
 'recall': 0.7598425196850394,
 'f1': 0.758116023760764}